In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
from scipy.signal import sawtooth
pi=np.pi
sin=np.sin
Ns=6
Nr=4
Nph=3
pp=Ns/Nph/2
p_peak=10**6
rpm=3000
f_base=rpm/60;
delta=round(2**10/Nph/Nr);
q_max=round(20000/f_base);

if q_max>400:
    q_max=400

circ_max=12
T_mech_rev=0.01


t_series_elec_revo=np.arange(0,T_mech_rev/Nr,T_mech_rev/delta/Nph/Nr)
t_series_mech_revo=np.arange(0,T_mech_rev,T_mech_rev/delta/Nph/Nr)


sp_series=np.arange(0,2*pi,2*pi/delta/Ns)

radial_wave=abs(sin(Ns/2*sp_series))

radial_pressure_elec_revo=np.array([])
radial_pressure_mech_revo=np.array([])

for j in np.arange(1,Nph+1):
    ph_A=(sawtooth(Ns/Nph*sp_series+2*(-j+Nph)*pi/Nph)+1)/2   
    radial_A=p_peak*(ph_A>float(Nph-1)/Nph)*radial_wave   #np.where(ph_A>float(Nph-1)/Nph,1,0)
    #radial_pressure_elec_revo=radial_A
    #radial_pressure_mech_revo=radial_pressure_elec_revo
    if j <Nph:
        for i in np.arange(1,round(len(t_series_elec_revo)/Nph)+1):
            radial_pressure_elec_revo=np.vstack((radial_pressure_elec_revo,radial_A)) if radial_pressure_elec_revo.size else radial_A
    else:
        
        for i in np.arange(1,len(t_series_elec_revo)-round(len(t_series_elec_revo)/Nph)*(Nph-1)+1):

            radial_pressure_elec_revo=np.vstack((radial_pressure_elec_revo,radial_A)) if radial_pressure_elec_revo.size else radial_A

for i in np.arange(1,Nr+1):
    
    radial_pressure_mech_revo=np.vstack((radial_pressure_mech_revo,radial_pressure_elec_revo)) if radial_pressure_mech_revo.size else radial_pressure_elec_revo


factor_over_time=abs(sin(t_series_mech_revo/T_mech_rev*Nph*Nr*pi));
factor_over_time=factor_over_time[:,np.newaxis]
matrix_over_time=np.array([])
    

for i in np.arange(1,len(sp_series)+1):

    matrix_over_time=np.hstack((matrix_over_time,factor_over_time)) if matrix_over_time.size else factor_over_time

radial_pressure_mech_revo=matrix_over_time*radial_pressure_mech_revo

In [ ]:
%matplotlib inline

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp2d

zfun=interp2d(sp_series,t_series_mech_revo,radial_pressure_mech_revo)
radial_surface_sparse=zfun(sp_series[::2**3],t_series_mech_revo[::2**3])

sp_plane,time_plane=np.meshgrid(np.arange(len(sp_series[::2**3])),np.arange(len(t_series_mech_revo[::2**3])))



fig = plt.figure(facecolor='w',figsize=(15,8))
ax = plt.axes(projection='3d')
ax.pbaspect = [1, 2,0.5]
xxx=sp_plane#/2/np.pi*360
yyy=time_plane
zzz=radial_surface_sparse
im=ax.plot_surface(xxx,yyy,zzz,rstride=1,cstride=1,alpha=0.5,cmap='jet',edgecolor='k')
ax.view_init(35,-106)
ax.set_xlabel(u'aa')
ax.set_zlim([0,p_peak*1.5])
plt.tight_layout()
#ax.autoscale('False')

#plt.colorbar(im,shrink=0.5)
plt.show()

In [3]:
import ipyvolume as ipv
from scipy.interpolate import interp2d

zfun=interp2d(sp_series,t_series_mech_revo,radial_pressure_mech_revo)
radial_surface_sparse=zfun(sp_series[::2**3],t_series_mech_revo[::2**3])

sp_plane,time_plane=np.meshgrid(np.arange(len(sp_series[::2**3])),np.arange(len(t_series_mech_revo[::2**3])))

xxx=sp_plane#/2/np.pi*360
yyy=time_plane
zzz=radial_surface_sparse

In [4]:
from matplotlib import cm
colormap = cm.coolwarm
znorm = zzz - zzz.min()
znorm /= znorm.ptp()
znorm.min(), znorm.max()
color = colormap(znorm)

In [31]:
ipv.figure()
ipv.style.use('dark')
ipv.plot_surface(xxx,yyy,zzz/100000,color=color[...,:3])
ipv.zlim(0, 20)
ipv.xlim(0, 128)
ipv.ylim(0, 128)
ipv.show()

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCDigKY=


In [26]:
xxx.max()

63

In [ ]:
ftimage=np.fft.fft2(zz)
ftimage=np.abs(np.fft.fftshift(ftimage))/ncol/nrow
plt.figure(figsize=(10,8))
plt.autoscale(enable=True, axis='x', tight=True)
plt.imshow(ftimage,interpolation='none',cmap=cm.jet,vmin=0.05)
plt.colorbar()

In [ ]:
plt.figure(figsize=(8,6))
plt.imshow(radial_surface_sparse,origin='zero',cmap=cm.jet)
ax=plt.gca()
ax.set_aspect('equal')
plt.colorbar()
plt.show()